Adapted from [MMIDAS repo](https://github.com/AllenInstitute/MMIDAS/blob/main/notebooks/smartseq_data_prep.ipynb).

### Preparing the data
In this notebook, we demonstrate how to prepare the Mouse Smart-seq dataset, which is a single-cell dataset was released as part of a transcriptomic cell types study in [Tasic et al., 2018](https://portal.brain-map.org/atlases-and-data/rnaseq/mouse-v1-and-alm-smart-seq). The dataset includes RNA sequencing of neurons from the anterolateral motor cortex (ALM) and primary visual cortex (VISp) regions of adult mice using Smart-seq (SSv4) platform. 

In [269]:
import pathlib

import pandas as pd
import numpy as np
import anndata as ad
from scipy.sparse import csr_matrix
import scipy.io as sio
from sklearn.preprocessing import normalize
from cplAE_MET.utils.analysis_tree_helpers import HTree, get_merged_types

Download ```zip``` files and locate them within the ```data``` folder. 

In [18]:
# toml_file = 'pyproject.toml'
# sub_file = 'smartseq_files'
# config = get_paths(toml_file=toml_file, sub_file=sub_file)
# data_path = config['paths']['main_dir'] / config['paths']['data_path']

data_path = pathlib.Path("../data/raw/mouse_VISp_gene_expression_matrices_2018-06-14")

In [13]:
# Load the mouse Smart-seq VISp data
data_VISp_exon = data_path / 'mouse_VISp_2018-06-14_exon-matrix.csv'
anno_VISp = data_path / 'mouse_VISp_2018-06-14_samples-columns.csv'
df_vis_exon = pd.read_csv(data_VISp_exon)
df_vis_anno = pd.read_csv(anno_VISp, encoding='unicode_escape')

# # Load the mouse Smart-seq ALM data
# data_ALM_exon = data_path / 'mouse_ALM_2018-06-14_exon-matrix.csv'
# anno_ALM = data_path / 'mouse_ALM_2018-06-14_samples-columns.csv'
# df_alm_exon = pd.read_csv(data_ALM_exon)
# df_alm_anno = pd.read_csv(anno_ALM, encoding='unicode_escape')

# print(f'Total number of cells in VISp and ALM: {len(df_vis_anno)}, {len(df_alm_anno)}')
print(f'Total number of cells in VISp and ALM: {len(df_vis_anno)}')

Total number of cells in VISp and ALM: 15413


In [17]:
# Get the neuronal cells across brain regions
vis_neuron = df_vis_anno['class'].isin(['GABAergic', 'Glutamatergic'])
# alm_neuron = df_alm_anno['class'].isin(['GABAergic', 'Glutamatergic'])
vis_counts = df_vis_exon.values[:, 1:][:, vis_neuron].T
# alm_counts = df_alm_exon.values[:, 1:][:, alm_neuron].T

# df_anno_ = pd.concat([df_vis_anno[vis_neuron], df_alm_anno[alm_neuron]], ignore_index=True)
# total_count = np.concatenate((vis_counts, alm_counts), axis=0)

df_anno_ = df_vis_anno[vis_neuron]
total_count = vis_counts

# Normalized counts values using LogCPM
# logCPM = logcpm(x=total_count, scaler=1e6)
logCPM = np.log1p(normalize(total_count, axis=1, norm='l1') * 1e6)
print(np.sum(logCPM, axis=1))

[30890.15859407 34090.13980254 35085.63428565 ... 25374.21782027
 29123.50220171 25566.69549158]


In [174]:
ref_genes_df = pd.read_csv("../data/raw/mouse_VISp_gene_expression_matrices_2018-06-14/mouse_VISp_2018-06-14_genes-rows.csv")
gene_df = pd.read_csv("../data/raw/good_genes_beta_score.csv")
genes = gene_df[gene_df["BetaScore"] > 0.4]["Gene"].values

In [175]:
gene_indx = [np.where(ref_genes_df.gene_symbol.values == gg)[0][0] for gg in genes]
log1p = logCPM[:, gene_indx]
# remove low quality cells and CR and Meis2 subclasses
mask = (df_anno_['cluster']!='Low Quality') & (df_anno_['cluster']!='CR Lhx5') & (df_anno_['cluster']!='Meis2 Adamts19')
df_anno_temp = df_anno_[mask].reset_index() 
log1p = log1p[mask, :]

Build an AnnData object for the dataloader. 

In [270]:
# load the tree.csv to obtain colors for t-types on the taxonomies
htree_file = pathlib.Path("../data/raw/mouse_VISp_gene_expression_matrices_2018-06-14/tree_Mouse_ALM-VISp_2018.csv")
treeObj = HTree(htree_file=htree_file)
ttypes = treeObj.child[treeObj.isleaf]
colors = treeObj.col[treeObj.isleaf]
df_anno = df_anno_temp.copy() #df_anno_temp.rename(columns={"seq_name": "sample_id", "class": "class_label"})

# rename two cell types according to the taxonomy
df_anno.loc[df_anno['cluster'] == 'L6b VISp Col8a1 Rprm', 'cluster'] = 'L6b Col8a1 Rprm'
df_anno.loc[df_anno['cluster'] == 'L6 CT ALM Nxph2 Sla', 'cluster'] = 'L6 CT Nxph2 Sla'

df_anno['cluster_color'] = np.array([colors[0]]*len(df_anno['cluster']))
for cluster in df_anno.cluster.unique():
    df_anno.loc[df_anno['cluster'] == cluster, 'cluster_color'] = colors[ttypes == cluster][0]

df_anno['cluster'] = np.array([c.strip() for c in df_anno['cluster']])

for nc in [40,50,60,70,80,90]:
    name = "merged_types_" + str(nc)
    merged_t, _, _ = get_merged_types(htree_file=htree_file,  
                                        cells_labels=df_anno["cluster"].to_numpy(copy = True), 
                                        num_classes=nc,
                                        ref_leaf=np.unique(df_anno["cluster"].to_numpy(copy = True)),
                                        node="n1")
    df_anno[f"merged_cluster_label_at{nc}"] = merged_t

In [180]:
# save data as AnnData object
sub_df = df_anno[['sample_name', 'sample_id', 'seq_batch', 'sex', 'brain_hemisphere', 'brain_region', 
                  'brain_subregion', 'class', 'subclass', 'cluster', 'cluster_color', 'confusion_score',
                 ] + [f"merged_cluster_label_at{nc}" for nc in [40,50,60,70,80,90]]]
adata = ad.AnnData(X=csr_matrix(log1p), obs=sub_df)
adata.var_names = genes
adata.obs_names = sub_df.sample_id.values   
adata.write_h5ad("../data/raw/smartseq_VISp_Tasic_2018")

/Users/ian.convy/miniconda3/envs/cpl/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/Users/ian.convy/miniconda3/envs/cpl/lib/python3.8/site-packages/anndata/_core/anndata.py:864: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [527128530, 527128536, 527128542, 527128548, 527128554]

    Inferred to be: integer

  names = self._prep_dim_index(names, "obs")


In [133]:
met = sio.loadmat("../data/raw/MET_M120x4_50k_4Apr23.mat")

In [231]:
df_mat = df_anno.copy()
df_mat["specimen_id"] = "SEQ_" + df_anno["sample_id"].astype("string")
df_mat["class"] = df_mat["class"].map({"GABAergic": "inh", "Glutamatergic": "exc"})
df_mat["class_id"] = df_mat["class"].map({"exc": 0, "inh": 1})
df_mat["cluster_label"] = df_mat["cluster"]
df_mat["platform"] = "smartseq"

In [264]:
new_met = {}
new_met["T_dat"] = np.concatenate([met["T_dat"], log1p])
for (key, array) in met.items():
    array = np.squeeze(array)
    if "__" not in key and key not in {"T_dat", "gene_ids", "E_features", "M_features"}:
        if key not in df_mat.columns:
            seq_array = np.full((len(df_mat),) + array.shape[1:], np.nan)
            print(f"'{key}' is missing.")
        else:
            seq_array = df_mat[key].to_numpy()
        new_array = np.concatenate([array, seq_array])
        if new_array.dtype == object:
            new_array = new_array.astype("str")
        new_met[key] = new_array
new_met["gene_ids"] = met["gene_ids"]
new_met["E_features"] = met["E_features"]
new_met["M_features"] = met["M_features"]

'E_dat' is missing.
'M_dat' is missing.
'soma_depth' is missing.
'hist_ax_de_api_bas' is missing.
'group' is missing.
'subgroup' is missing.
'cluster_id' is missing.


In [268]:
sio.savemat("../data/raw/MET_M120x4_50k_4Apr23_wTasic.mat", new_met)